In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import sys
# sys.path.append("../source/")
sys.path.append("../")

In [ ]:
from source.analysis_viz import AgentInfo, WorldModelInfo, ProgressTracker

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import os

In [ ]:
import torch
import numpy as np

In [ ]:
root_dir = Path("/envroot/trainings/camera_train")
cmd_dir = root_dir / "collected_commands"
pos_dir = root_dir / "collected_positions"
embed_dir = root_dir / "collected_embeds"
img_dir = root_dir / "collected_imgs"

wm_dir = root_dir / "world_models"
ag_dir = root_dir / "agents"

In [ ]:
# prog_file = root_dir / "progress_model_names.txt"
# with open(prog_file, "r") as f:
#     models, timestamps = list(zip(*[map(str.strip, l.strip().split("@")) for l in f]))

In [ ]:
print(ProgressTracker(root_dir))

In [ ]:
ag = AgentInfo(root_dir, "ag_00_00")
print(ag)
pos, cmd, embed, reward = ag.load_collected_data(0)

len(pos), len(cmd), len(embed), len(reward)
# remove the last postion, where the reward is not used to guide the camera
# pos = [p[:-1] for p in pos]

In [ ]:
wm = WorldModelInfo(root_dir, ag.info_dict["restart_00"]["parent_model"])
wm

In [ ]:
# action_rewards = [target_predictor(embed[i].to(device), torch.tensor(pos[i][:-1]).to(device, dtype=torch.float32)) for i in range(len(pos))]
action_rewards = torch.vstack(reward).numpy()
rewards = np.max(action_rewards, axis=1)

In [ ]:
target_embeds = [torch.mean(embed[i], axis=1) for i in range(len(embed))]

In [ ]:
from source.analysis_viz import scale_pca_tsne_transform
embed_tsne, embed_pca = scale_pca_tsne_transform(torch.vstack(target_embeds))

In [ ]:
pos = [p[:-1] for p in pos]
arr_pos = np.vstack(pos)

In [ ]:
plt.figure(figsize=(8, 6))
vmin = np.quantile(rewards, 0.1)
vmax = np.quantile(rewards, 0.9)
plt.scatter(embed_tsne[:, 0], embed_tsne[:, 1], s=2,
              c=rewards, cmap="jet", vmin=vmin, vmax=vmax)
plt.gca().set_aspect("equal")
plt.xlabel("tSNE axis-1")
plt.ylabel("tSNE axis-2")
cbar = plt.colorbar()
cbar.ax.set_ylabel('Reward')
# plt.show()
plt.savefig("tsne.png")
# ax[1].legend()
# ax[1].set_xlim([-80, 80])
# ax[1].set_ylim([-80, 80])

fig, ax = plt.subplots(2, 1, figsize=(8, 6))
# for p in pos:
    # ax.scatter(p[:, 0], p[:, 1], s=1)
# ax.scatter(pos[:, 0], pos[:, 1], s=pos[:, 2] / 100)
i = 1
im = ax[i].scatter(arr_pos[:, 0], arr_pos[:, 1], c=np.log10(arr_pos[:, 2]), s=2)
ax[i].set_xlim([-185, 185])
ax[i].set_ylim([-95, 5])
ax[i].set_xlabel("Pan (deg)")
ax[i].set_ylabel("Tilt (deg)")
ax[i].set_aspect("equal")
cbar = fig.colorbar(im)
cbar.ax.set_ylabel('$\log_{10}$ Zoom')
i = 0
im = ax[i].scatter(arr_pos[:, 0], arr_pos[:, 1], c=rewards, s=2, cmap="jet", vmin=vmin, vmax=vmax)
ax[i].set_xlim([-185, 185])
ax[i].set_ylim([-95, 5])
ax[i].set_xlabel("Pan (deg)")
ax[i].set_ylabel("Tilt (deg)")
ax[i].set_aspect("equal")
cbar = fig.colorbar(im)
cbar.ax.set_ylabel('Reward')
fig.savefig("pointings.png")
# plt.show()
# fig.savefig("pointings.png")

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(8, 4))
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(projection='3d')
# for p in pos:
    # ax.scatter(p[:, 0], p[:, 1], s=1)
im = ax.scatter(arr_pos[:, 0], arr_pos[:, 2], arr_pos[:, 1], s=2, c=rewards, cmap="jet", vmin=vmin, vmax=vmax)
# ax.set_xlim([-185, 185])
# ax.set_ylim([-95, 5])
ax.set_xlabel("Pan (deg)")
ax.set_zlabel("Tilt (deg)")
ax.set_ylabel("Zoom")
fig.colorbar(im)
# ax.set_aspect("equal")
# ax.set_ylim(700, 0)
# fig.savefig("pointings.png")

In [ ]:
# sel = (arr_pos[:, 0] < -0) & (arr_pos[:, 0] > -15) #& (pos[:, 1] < -65) & (pos[:, 1] > -75)
# print(pos[sel], np.vstack(cmd)[sel])

# plt.plot(rewards[sel])
# plt.ylabel("Reward")
# plt.xlabel("Number of movement")

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
%matplotlib agg
pp = PdfPages('all_pointings_output.pdf')
li_rewards = rewards.reshape(-1, 30)
for j in range(len(pos)):
    fig, ax = plt.subplots(2, 1, figsize=(8, 6))
    i = 0
    im = ax[i].scatter(pos[j][:, 0], pos[j][:, 1], c=li_rewards[j], s=2, cmap="jet", vmin=vmin, vmax=vmax)
    ax[i].set_xlim([-185, 185])
    ax[i].set_ylim([-95, 5])
    ax[i].set_xlabel("Pan (deg)")
    ax[i].set_ylabel("Tilt (deg)")
    ax[i].set_aspect("equal")
    cbar = fig.colorbar(im)
    cbar.ax.set_ylabel('Reward')
    i = 1
    ax[i].plot(li_rewards[j])
    ax[i].set_ylabel("Reward")
    ax[i].set_xlabel("Number of movement")
    pp.savefig(fig)
    # plt.plot(rewards[sel])
    # plt.ylabel("Reward")
    # plt.xlabel("Number of movement")
pp.close()

In [ ]:
# import matplotlib.animation as animation

# # pos = []
# # for fn in fnames:
# #     with open(pos_dir / f"{fn}.txt", "r") as f:
# #         pos.append(np.array([l.strip().split(",") for l in f.readlines()], dtype=float))
# # pos = np.vstack(pos)

# fig, ax = plt.subplots(1, 1, figsize=(8, 4))
# ax.set_xlim([-185, 185])
# ax.set_ylim([-95, 5])
# ax.set_xlabel("Pan (deg)")
# ax.set_ylabel("Tilt (deg)")
# ax.set_aspect("equal")
# scat = ax.scatter(pos[0, 0], pos[0, 1], s=5, c='tab:red')


# def animate(i):
#     # if i < 5:
#     #     for j in range(i-1):
#     #         scat.plot(pos[j, 0], pos[j, 1], s=5)
#     # else:
#     # scat.set_offsets((pos[i, 0], pos[i, 1]))
#     # for j in range(max(i-5, 0), i):
#     # # if i > 0:
#     ax.scatter(pos[i, 0], pos[i, 1], s=5, c='tab:red')
#     return scat,

# # def update(frame):
# #     global positions
# #     positions += velocities  # Update positions
# #     lo = 0 if frame < 10 else frame - 10
# #     for i in range(lo, frame):
# #         scat.set_offsets(positions[i])
# #     # Update the trails
# #     for i in range(num_points):
# #         trails[i].append(positions[i].copy())
# #         if len(trails[i]) > 10:  # Limit trail length
# #             trails[i].pop(0)

# #     # Update the scatter plot
# #     scatter.set_offsets(positions)

# #     # Draw the trails
# #     for trail in trails:
# #         if len(trail) > 1:
# #             trail_array = np.array(trail)
# #             ax.plot(trail_array[:, 0], trail_array[:, 1], 'b-', alpha=0.5)

# #     return scatter,


# ani = animation.FuncAnimation(fig, animate, repeat=True,
#                                     frames=len(pos) - 1, interval=20)
# # plt.show()
# writer = animation.PillowWriter(fps=15,
#                                 metadata=dict(artist='Me'),
#                                 bitrate=1800)
# ani.save('scatter.gif', writer=writer)